## GeoCat experimental

In [1]:
import importlib
from collections import defaultdict
import numpy as np
import pandas as pd
import recommenders as rec
import geocat
import networkx as nx
import matplotlib.pyplot as plt
import areamanager
import math
import timeit
from IPython.core.debugger import set_trace
%matplotlib inline
def string_to_array(string):
    if string == '':
        return list()
    return eval(string)
importlib.reload(geocat)
importlib.reload(rec)
CITY="madison"



In [2]:
df_checkin=pd.read_csv("../data/checkin/"+CITY+".csv",converters={'categories':string_to_array})

df_checkin_train=pd.read_csv("../data/checkin/train/"+CITY+".csv",converters={'categories':string_to_array})


users_id=df_checkin['user_id'].drop_duplicates().reset_index(drop=True)
pois_id=df_checkin['business_id'].drop_duplicates().reset_index(drop=True)
user_num=len(users_id)
poi_num=len(pois_id)


df_checkin_train=df_checkin_train.set_index('user_id')

df_checkin_train.head()

,business_id,date,latitude,longitude,categories,subarea_id
user_id,,,,,,
0,1744,2017-12-15 14:05:01,43.075012,-89.447498,"[desserts, cafes, coffee]",2143.0
0,2746,2018-01-18 20:57:23,43.106774,-89.497738,"[desserts, tradamerican, diners, gastropubs, b...",2641.0
0,30,2018-01-23 23:14:37,43.072654,-89.384062,"[bars, mexican]",2133.0
0,432,2018-01-27 02:17:06,43.072439,-89.384610,"[tradamerican, bars]",2133.0
0,22,2018-01-27 02:31:26,43.075100,-89.381069,[mexican],2132.0


In [3]:
df_checkin_test=pd.read_csv("../data/checkin/test/"+CITY+".csv",converters={'categories':string_to_array})

In [4]:
ground_truth = defaultdict(set)
for index,row in df_checkin_test[['user_id','business_id']].drop_duplicates().iterrows():
    ground_truth[row['user_id']].add(row['business_id'])

In [5]:
poi_coos = {}
for lid,latlon in df_checkin[['business_id','latitude','longitude']].reset_index(drop=True).drop_duplicates().set_index('business_id').iterrows():
    poi_coos[lid] = tuple(latlon)

In [6]:
poi_cats = {}
for lid,row in df_checkin[['business_id','categories']].reset_index(drop=True).drop_duplicates(subset=['business_id']).set_index('business_id').iterrows():
    poi_cats[lid] = row.categories

In [7]:
dict_alias_title,category_tree,dict_alias_depth=geocat.category_manipulation_utils()
undirected_category_tree=category_tree.to_undirected()

In [8]:
training_matrix = np.zeros((user_num, poi_num))
for user_id,poi_id in df_checkin_train.business_id.iteritems():
   # print(user_id,poi_id)
    training_matrix[user_id,poi_id]+=1

In [9]:
all_uids = list(range(user_num))
all_lids = list(range(poi_num))

In [10]:
np.random.shuffle(all_uids)

In [115]:
user_predict={}
for index,row in pd.read_csv("../data/result/sigir11_top_100.txt",header=None).iterrows():
    user_id=row[0]
    lid_list=row[1]
    user_predict[user_id]=eval(lid_list)

In [12]:
df_poi_neighbor=pd.read_csv("../data/neighbor/"+CITY+".csv",converters={'neighbors':string_to_array})
df_poi_neighbor=df_poi_neighbor.set_index("business_id")
poi_neighbors={}
for lid,row in df_poi_neighbor.iterrows():
    neighbors=row['neighbors']
    poi_neighbors[lid]=neighbors
df_poi_neighbor=None

In [130]:
import objfunction
importlib.reload(objfunction)

<module 'objfunction' from '/home/heitor/recsys/algorithms/objfunction.py'>

In [131]:
N=80
K=20
range_K=range(K)
for cnt, uid in enumerate(all_uids):
    if uid in ground_truth:
        tmp_rec_list=user_predict[uid][:N]
        rec_list=[]
        
        lids=training_matrix[uid].nonzero()[0]
        lid_visits=training_matrix[:,lids].sum(axis=0)
        mean_visits=lid_visits.mean()
        relevant_lids=lids[lid_visits>mean_visits]
        relevant_cats=set()
        for lid in relevant_lids:
            relevant_cats.update(poi_cats[lid])
        # log_size=training_matrix[0,training_matrix[0,:].nonzero()[0]].sum()
        user_log=training_matrix[uid]
        
        
        log_poi_ids=list()
        poi_cover=list()
        for lid in user_log.nonzero()[0]:
            for visits in range(int(user_log[lid])):
                poi_cover.append(0)
                log_poi_ids.append(lid)
                
        assert user_log[user_log.nonzero()[0]].sum() == len(poi_cover)
#         print(uid)
#         print("Count:",cnt)
        
            
        for i in range_K:
#             print(i)
            for j in range(len(tmp_rec_list)):
                candidate_poi_id=tmp_rec_list[j]
#                 print(objfunction.gc(candidate_poi_id,relevant_cats,poi_cats))
#                 print(objfunction.min_dist_to_list_cat(candidate_poi_id,rec_list,poi_cats,undirected_category_tree))
                print(objfunction.update_geo_cov(candidate_poi_id,log_poi_ids,K,poi_cover.copy(),poi_neighbors))
                pass
        

NameError: name 'log_pois_ids' is not defined

In [ ]:
N=80
K=20
VERY_SMALL_VALUE=-100
i=0

# tmp_rec_list=rec.mostpopular(df_checkin,K)
# tmp_rec_list['score']=tmp_rec_list['score']/tmp_rec_list['score'].max()
# print(tmp_rec_list)
dict_rec_list=dict()
for user_id in users:
    #user_id="scaxsfOynKUZEijn_UCTkg"
    #print(user_id)
    # User visits
    df_user_checkin=df_checkin_train.loc[[user_id]].reset_index(drop=True)
    
    ####print(df_user_checkin['business_id'])
    
    list_user_pois=set(df_user_checkin['business_id'].drop_duplicates().values)
    
    
    tmp_rec_list=rec.mostpopular(df_checkin_train.reset_index(),N,num_users,list_user_pois)
    
    # add other columns
    tmp_rec_list=pd.merge(tmp_rec_list,df_poi.reset_index(),on='business_id').drop_duplicates(subset=df_poi.reset_index().columns.difference(['categories'])).reset_index(drop=True)
    
    
    business_cover=list()
    pois_id=set()
    for business_id in df_user_checkin['business_id'].reset_index(drop=True):
        business_cover.append(0)
        pois_id.add(business_id)

    # Constants
    #COVER_OF_POI=len(df_user_review)/len(tmp_rec_list)
    div_geo_cat_weight = 0.75 # beta,this is here because of the work to be done on parameter customization for each user
    div_weight = 0.5 # lambda, geo vs cat
    current_proportionality=0
    
    rec_list=pd.DataFrame(columns=['business_id','score','latitude','longitude','categories'])
#     relevant_categories_user=geocat.relevant_categories_to_the_user(df_user_checkin)

    # REPLICATE WRONG CODE
    series_user_checkin=df_user_checkin.groupby('business_id').count()['date']
    series_user_checkin.name="checkin"

    
    # This code is to replicate the error in original geocat
    mean_poi_visits=(series_poi_checkin[pois_id]*series_user_checkin).sum()/series_user_checkin.sum()
    # using entry from java
    #####mean_poi_visits=pd.read_csv("~/checkinjava.csv").set_index("business_id")['checkin'].mean()
    # Relevant categories

    relevant_categories = set()
    print(mean_poi_visits)
    #pd.read_csv("~/checkinjava.csv").set_index("business_id")['checkin']
    ####for poi_id,checkins in series_poi_checkin[business_cover.keys()].iteritems():
    
    for poi_id,checkins in series_poi_checkin[pois_id].iteritems():
        # Check if poi is relevant
        if checkins >= mean_poi_visits:
            # add relevant categories
#             print(poi_id)
#             print(df_poi.loc[poi_id]['categories'])
            for category in df_poi.loc[poi_id]['categories']:
                relevant_categories.add(category)
        
        

    print("Number of relevant categories:",len(relevant_categories))#,"Number of relevant cat dupl:",cc)
    # REPLICATE WRONG CODE
    #for i in relevant_categories:
   #     print(dict_alias_title[i])
    #print(relevant_categories)

    
    # post-processing for diversification
    for i in range(K):
#         start = timeit.default_timer()
        #num_current_candidates=len(tmp_rec_list)
       # set_trace()
        poi_to_insert=None
        max_objective_value=VERY_SMALL_VALUE
        rec_list_categories=geocat.category_in_rec_list(tmp_rec_list)
        print(i)
        
        for index,poi_candidate in tmp_rec_list.iterrows():
            candidate_neighbors=df_poi_neighbor.loc[poi_candidate.business_id].neighbors
            
            
            poi_neighbors=df_poi.loc[set(candidate_neighbors) & list_user_pois]
            #print(len(poi_neighbors))
            
            objective_value=geocat.objective_ILD_GC_PR(poi_candidate,df_user_checkin,rec_list,K,\
                                                business_cover,current_proportionality,\
                                               div_geo_cat_weight,div_weight,dict_alias_title,undirected_category_tree,\
                                                      relevant_categories,rec_list_categories,poi_neighbors)
            
            if objective_value > max_objective_value:
                max_objective_value=objective_value
                poi_to_insert=poi_candidate 
        
#         stop = timeit.default_timer()
#         print('Time:', stop - start)
        
        if poi_to_insert is not None:
            candidate_neighbors=df_poi_neighbor.loc[poi_to_insert.business_id].neighbors
            poi_neighbors=df_poi.loc[set(candidate_neighbors) & list_user_pois]
            poi_to_insert['score']=max_objective_value
            rec_list=rec_list.append(poi_to_insert)
            # remove from tmp_rec_list
            tmp_rec_list=tmp_rec_list[tmp_rec_list['business_id']!=poi_to_insert['business_id']].reset_index(drop=True)
            
            current_proportionality=geocat.update_geo_cov(poi_to_insert,df_user_checkin,K,business_cover,poi_neighbors)
            
            
    
    dict_rec_list[user_id]=rec_list[['business_id','score']]
    print(rec_list[['business_id','score']].sort_values(by="score",ascending=False).reset_index(drop=True))
    
    i+=1
    if i % num_users==0:
        print(i)


In [ ]:
pd.read_csv("~/teste.csv",header=None)

In [ ]:
dict_alias_title['']

In [ ]:
pd.merge(pd.read_csv("~/a.txt",header=None),
pd.read_csv("~/b.txt",header=None),how='inner')